<a href="https://colab.research.google.com/github/deysi-caraballo/tfm/blob/main/TFM_vertexAI_automl_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the Lice`nse is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI: Create, train, and deploy an AutoML text classification model

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
 <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This notebook walks you through the major phases of building and using an AutoML text classification model on [Vertex AI](https://cloud.google.com/vertex-ai/docs/).

Learn more about [Classification for text data](https://cloud.google.com/vertex-ai/docs/training-overview#classification_for_text).

### Objective

In this tutorial, you learn how to use `AutoML` to train a text classification model.

This tutorial uses the following Google Cloud ML services:

- `AutoML Training`
- `Vertex AI Model` resource

The steps performed include:

* Create a `Vertex AI Dataset`.
* Train an `AutoML` text classification `Model` resource.
* Obtain the evaluation metrics for the `Model` resource.
* Create an `Endpoint` resource.
* Deploy the `Model` resource to the `Endpoint` resource.
* Make an online prediction
* Make a batch prediction

### Dataset

In this notebook, you use the "Happy Moments" sample dataset to train a model. The resulting model classifies happy moments into categores that reflect the causes of happiness.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI Training and Serving
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage

### Installation

Install the following packages for executing this notebook.

In [1]:
# install packages
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                    google-cloud-storage \
                                    jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00


### Colab Only: Uncomment the following cell to restart the kernel

In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Before you begin

#### Set your project ID

**If you don't know your project ID**, try the following:
-  Run `gcloud config list`
-  Run `gcloud projects list`
-  See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [1]:
PROJECT_ID = '800463162730'  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI.
Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [16]:
REGION = "europe-west4"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
- Do nothing as you are already authenticated.

**2. Local JupyterLab Instance,** uncomment and run.

In [3]:
import os
import google.auth

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/sa.json'

In [4]:
!export GOOGLE_APPLICATION_CREDENTIALS=/content/sa.json

In [5]:
!gcloud auth login --cred-file=$GOOGLE_APPLICATION_CREDENTIALS


Authenticated with service account credentials for: [tfm-pasalera@snappy-guard-389922.iam.gserviceaccount.com].
Your current project is [800463162730].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


**3. Colab,** uncomment and run:

In [ ]:
#from google.colab import auth
#auth.authenticate_user()

**4. Service Account or other**
- See all the authentication options here: [Google Cloud Platform Jupyter Notebook Authentication Guide](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [7]:
BUCKET_NAME = 'deysi-tfm-nlp-bucket'  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
#! gsutil mb -l {REGION} {BUCKET_URI}

### Import libraries and define constants

In [8]:
import jsonlines
from google.cloud import aiplatform, storage
from google.cloud.aiplatform import jobs

### Initialize Vertex AI

Initialize the Vertex AI SDK for Python for your project.

In [17]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## Create a `Dataset` resource and import your data

The notebook uses the 'Happy Moments' dataset for demonstration purposes. You can change it to another text classification dataset that [conforms to the data preparation requirements](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text#classification).

Using the Python SDK, you create a dataset and import the dataset in one call to `TextDataset.create()`, as shown in the following cell.

Creating and importing data is a long-running operation. This next step can take a while. The `create()` method waits for the operation to complete, outputting statements as the operation progresses. The statements contain the full name of the dataset that you use in the following section.

**Note**: You can close the noteboook while you wait for this operation to complete.

In [18]:
# Use a timestamp to ensure unique resources
src_uris = "gs://deysi-tfm-nlp-bucket/tablas_vertex/tfm-preguntas-crudo-aumentado.csv"
display_name = "dataset-crudo-aumentado"

text_dataset = aiplatform.TextDataset.create(
    display_name=display_name,
    gcs_source=src_uris,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
    sync=True,
)

Creating TextDataset


INFO:google.cloud.aiplatform.datasets.dataset:Creating TextDataset


Create TextDataset backing LRO: projects/800463162730/locations/europe-west4/datasets/3930949782368944128/operations/5095172342363979776


INFO:google.cloud.aiplatform.datasets.dataset:Create TextDataset backing LRO: projects/800463162730/locations/europe-west4/datasets/3930949782368944128/operations/5095172342363979776


TextDataset created. Resource name: projects/800463162730/locations/europe-west4/datasets/3930949782368944128


...

INFO:google.cloud.aiplatform.datasets.dataset:Importing TextDataset data: projects/800463162730/locations/europe-west4/datasets/3930949782368944128


Import TextDataset data backing LRO: projects/800463162730/locations/europe-west4/datasets/3930949782368944128/operations/4942049955033382912


INFO:google.cloud.aiplatform.datasets.dataset:Import TextDataset data backing LRO: projects/800463162730/locations/europe-west4/datasets/3930949782368944128/operations/4942049955033382912


TextDataset data imported. Resource name: projects/800463162730/locations/europe-west4/datasets/3930949782368944128


INFO:google.cloud.aiplatform.datasets.dataset:TextDataset data imported. Resource name: projects/800463162730/locations/europe-west4/datasets/3930949782368944128


## Train your text classification model

Now you can begin training your model. Training the model is a two part process:

1. **Define the training job.** You must provide a display name and the type of training you want when you define the training job.
2. **Run the training job.** When you run the training job, you need to supply a reference to the dataset to use for training. At this step, you can also configure the data split percentages.

You do not need to specify [data splits](https://cloud.google.com/vertex-ai/docs/general/ml-use). The training job has a default setting of  training 80%/ testing 10%/ validate 10% if you don't provide these values.

To train your model, you call `AutoMLTextTrainingJob.run()` as shown in the following snippets. The method returns a reference to your new `Model` object.

As with importing data into the dataset, training your model can take a substantial amount of time. The client library prints out operation status messages while the training pipeline operation processes. You must wait for the training process to complete before you can get the resource name and ID of your new model, which is required for model evaluation and model deployment.

**Note**: You can close the notebook while you wait for the operation to complete.

In [19]:
# Define the training job
training_job_display_name = "tfm-clasificacion-training-job-automl-preguntas-crudas"
job = aiplatform.AutoMLTextTrainingJob(
    display_name=training_job_display_name,
    prediction_type="classification",
    multi_label=False,
)

In [ ]:
model_display_name = "tfm-clasificacion-automl-preguntas-crudas"

# Run the training job
model = job.run(
    dataset=text_dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.1,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    sync=True,
)

View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/719615516972417024?project=800463162730


INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/719615516972417024?project=800463162730


AutoMLTextTrainingJob projects/800463162730/locations/europe-west4/trainingPipelines/719615516972417024 current state:
PipelineState.PIPELINE_STATE_PENDING


INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/800463162730/locations/europe-west4/trainingPipelines/719615516972417024 current state:
PipelineState.PIPELINE_STATE_PENDING


AutoMLTextTrainingJob projects/800463162730/locations/europe-west4/trainingPipelines/719615516972417024 current state:
PipelineState.PIPELINE_STATE_PENDING


...

INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/800463162730/locations/europe-west4/trainingPipelines/719615516972417024 current state:
PipelineState.PIPELINE_STATE_RUNNING


AutoMLTextTrainingJob projects/800463162730/locations/europe-west4/trainingPipelines/719615516972417024 current state:
PipelineState.PIPELINE_STATE_RUNNING


INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/800463162730/locations/europe-west4/trainingPipelines/719615516972417024 current state:
PipelineState.PIPELINE_STATE_RUNNING


AutoMLTextTrainingJob projects/800463162730/locations/europe-west4/trainingPipelines/719615516972417024 current state:
PipelineState.PIPELINE_STATE_RUNNING


INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/800463162730/locations/europe-west4/trainingPipelines/719615516972417024 current state:
PipelineState.PIPELINE_STATE_RUNNING


## Review model evaluation scores

After your model training has finished, you can review the evaluation scores for it using the `list_model_evaluations()` method. This method will return an iterator for each evaluation slice.

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

## Deploy your text classification model

Once your model has completed training, you must deploy it to an _endpoint_ to get online predictions from it. When you deploy the model to an endpoint, a copy of the model is made on the endpoint with a new resource name and display name.

You can deploy multiple models to the same endpoint and split traffic between the various models assigned to the endpoint. However, you must deploy one model at a time to the endpoint. To change the traffic split percentages, you must assign new values on your second (and subsequent) models each time you deploy a new model.

The following code block demonstrates how to deploy a model. The code snippet relies on the Python SDK to create a new endpoint for deployment. The call to `modely.deploy()` returns a reference to an `Endpoint` object--you need this reference for online predictions in the next section.

In [ ]:
deployed_model_display_name = "tfm-clasificacion-automl-preguntas-crudas"

endpoint = model.deploy(
    deployed_model_display_name=deployed_model_display_name, sync=True
)

## Get online predictions from your model

Now that you have your endpoint, you can get online predictions from the text classification model. To get the online prediction, you send a prediction request to your endpoint.

In [ ]:
content = "¿Subirán las pensiones durante el año 2023?"

response = endpoint.predict(instances=[{"content": content}])

for prediction_ in response.predictions:
    ids = prediction_["ids"]
    display_names = prediction_["displayNames"]
    confidence_scores = prediction_["confidences"]
    for count, id in enumerate(ids):
        print(f"Prediction ID: {id}")
        print(f"Prediction display name: {display_names[count]}")
        print(f"Prediction confidence score: {confidence_scores[count]}")